In [641]:
import pandas as pd
import numpy as np
import os

In [642]:
def clean_excel(team):
    team_df = pd.read_csv(f'Resources/2020updates/{team}.csv')
    team_df = team_df.drop(columns=['Unnamed: 3', 'Unnamed: 4', '1stD', '1stD.1', 'Offense', 'Defense', 'Sp. Tms'])
    team_df = team_df.rename(columns={'Unnamed: 5': 'Results', 
                                      'OT': 'Entering Wins', 
                                      'Unnamed: 8': 'Home', 
                                      'Tm': 'PF', 
                                      'Opp.1': 'PA', 
                                      'TotYd': 'OTotYd', 
                                      'PassY': 'OPassY', 
                                      'RushY': 'ORushY', 
                                      'TO': 'TO_lost', 
                                      'TotYd.1': 'DTotYd', 
                                      'PassY.1': 'DPassY', 
                                      'RushY.1': 'DRushY', 
                                      'TO.1': 'TO_won'})
    team_df['week_after_bye'] = ''
    return team_df

In [643]:
def clean_yearlies(df, base_team=''):
    
    # Add a 'base team' column
    df['Base_Team'] = base_team
    
    # Unify the data so all null rows are set to zero
    df['PF'].fillna(0, inplace = True)
    df['PA'].fillna(0, inplace = True)
    df['OPassY'].fillna(0, inplace = True)
    df['ORushY'].fillna(0, inplace = True)
    df['TO_lost'].fillna(0, inplace = True)
    df['DPassY'].fillna(0, inplace = True)
    df['DRushY'].fillna(0, inplace = True)
    df['TO_won'].fillna(0, inplace = True)
    df['OTotYd'].fillna(0, inplace = True)
    df['DTotYd'].fillna(0, inplace = True)

    # Caluculate the cumulative sum of each stat by week, EXCLUDING that week's numbers. Should reflect the
    # team's states going into the game, not coming out of it.
    df['PF Tally'] = df['PF'].cumsum() - df['PF']
    df['PA Tally'] = df['PA'].cumsum() - df['PA']
    df['OPassY Tally'] = df['OPassY'].cumsum() - df['OPassY']
    df['ORushY Tally'] = df['ORushY'].cumsum() - df['ORushY']
    df['TO_lost Tally'] = df['TO_lost'].cumsum() - df['TO_lost']
    df['TO_won Tally'] = df['TO_won'].cumsum() - df['TO_won']
    df['DPassY Tally'] = df['DPassY'].cumsum() - df['DPassY']
    df['DRushY Tally'] = df['DRushY'].cumsum() - df['DRushY']
    df['OTotYd Tally'] = df['OTotYd'].cumsum() - df['OTotYd']
    df['DTotYd Tally'] = df['DTotYd'].cumsum() - df['DTotYd']
    
    # Clean 'Results' column so wins have a value of 1 and losses/ties have a value of 0
    df['Results'] = df['Results'].replace(['L', 'T'], 0)
    df['Results'] = df['Results'].replace('W', 1)
    df['Results'].fillna(0, inplace = True)

    # Caluculate win tallies going into each game
    df = df.rename(columns={"Entering Wins": "Wins Tally"})
    df['Wins Tally'] = 0
    df['Wins Tally'] = df['Results'].cumsum() - df['Results']
    
    # Clean 'week_after_bye' column so only the 'Bye Week' row and the row after it have a value of 1.
    # 'Bye Week' row will be dropped later.
    df['week_after_bye'] = 0
    df.loc[df['Opp'] == 'Bye Week', 'week_after_bye'] = 1
    df['after_bye'] = df['week_after_bye'].cumsum()
    df['week_after_bye'].values[df['week_after_bye'] >= 2] = 0
    df['week_after_bye'] = df['after_bye'].cumsum() - df['after_bye']
    df['after_bye'].values[df['week_after_bye'] >= 2] = 0
    
    df = df.drop(columns=['week_after_bye', 'Rec', 'Date', 'PF', 'PA', 'OPassY', 'ORushY', 'TO_lost', 'TO_won', 'DPassY', 'DRushY', 'OTotYd', 'DTotYd'])
    
    # Delete 'Bye' weeks and change column name
    df = df[df['Opp'] != 'Bye Week']

    # Clean 'Home' column so home games have a value of 1 and away games have a value of 0
    df['Home'] = df['Home'].replace(['@', 'N'], 0)
    df['Home'].fillna(1, inplace = True)
    
    return df

In [644]:
def yearly_data(df, year, HC2_range=0, QB2_range=0, HC='', HC2='none', OC='', DC='', QB='', QB2='none'):
    df['year'] = year
    df = clean_yearlies(df)
    df['Head_Coach'] = HC
    if HC2 == 'none':
        pass
    else:
        df.loc[df['Week'] == HC2_range, 'Head_Coach'] = HC2
    df['OC'] = OC
    df['DC'] = DC
    df['QB'] = QB
    if QB2 == 'none':
        pass
    else:
        df.loc[df['Week'] == QB2_range, 'QB'] = QB2
        
    df = df[df['Week'] <= 17]
    
    df['Year'] = df['year']
    df['After_Bye'] = df['after_bye']
    df = df.drop(columns=['year','after_bye'])
    
    df = df[['Year', 'Week', 
            'Day', 'Base_Team', 
            'Opp', 'Home', 
            'After_Bye', 'Wins Tally', 
            'PF Tally', 'PA Tally', 
            'OPassY Tally', 'ORushY Tally',
            'OTotYd Tally', 'TO_lost Tally', 
            'DPassY Tally', 'DRushY Tally', 
            'DTotYd Tally', 'TO_won Tally',
            'Head_Coach', 'OC', 'DC', 'QB', 
            'Results']]
        
    return df

In [645]:
def clean_team_names(df):
    # Reduce team names to just the mascot, since some teams have changed cities and might be mistaken for two separate teams
    df['Opp'] = df['Opp'].replace(['Los Angeles Rams', 'St. Louis Rams'], 'Rams')
    df['Opp'] = df['Opp'].replace(['Los Angeles Chargers', 'San Diego Chargers'], 'Chargers')
    df['Opp'] = df['Opp'].replace('Seattle Seahawks', 'Seahawks')
    df['Opp'] = df['Opp'].replace('Arizona Cardinals', 'Cardinals')
    df['Opp'] = df['Opp'].replace('Green Bay Packers', 'Packers')
    df['Opp'] = df['Opp'].replace('New Orleans Saints', 'Saints')
    df['Opp'] = df['Opp'].replace('New York Giants', 'Giants')
    df['Opp'] = df['Opp'].replace('Chicago Bears', 'Bears')
    df['Opp'] = df['Opp'].replace('Carolina Panthers', 'Panthers')
    df['Opp'] = df['Opp'].replace('Tampa Bay Buccaneers', 'Buccaneers')
    df['Opp'] = df['Opp'].replace('Atlanta Falcons', 'Falcons')
    df['Opp'] = df['Opp'].replace(['Washington Redskins', 'Washington Football Team'], 'Washington')
    df['Opp'] = df['Opp'].replace('Baltimore Ravens', 'Ravens')
    df['Opp'] = df['Opp'].replace('Philadelphia Eagles', 'Eagles')
    df['Opp'] = df['Opp'].replace('Dallas Cowboys', 'Cowboys')
    df['Opp'] = df['Opp'].replace('Kansas City Chiefs', 'Chiefs')
    df['Opp'] = df['Opp'].replace('Detroit Lions', 'Lions')
    df['Opp'] = df['Opp'].replace('Minnesota Vikings', 'Vikings')
    df['Opp'] = df['Opp'].replace('Cincinnati Bengals', 'Bengals')
    df['Opp'] = df['Opp'].replace(['Oakland Raiders', 'Las Vegas Raiders'], 'Raiders')
    df['Opp'] = df['Opp'].replace('Pittsburgh Steelers', 'Steelers')
    df['Opp'] = df['Opp'].replace('Cleveland Browns', 'Browns')
    df['Opp'] = df['Opp'].replace('Denver Broncos', 'Broncos')
    df['Opp'] = df['Opp'].replace('Indianapolis Colts', 'Colts')
    df['Opp'] = df['Opp'].replace('Houston Texans', 'Texans')
    df['Opp'] = df['Opp'].replace('New York Jets', 'Jets')
    df['Opp'] = df['Opp'].replace('Jacksonville Jaguars', 'Jaguars')
    df['Opp'] = df['Opp'].replace('Miami Dolphins', 'Dolphins')
    df['Opp'] = df['Opp'].replace('New England Patriots', 'Patriots')
    df['Opp'] = df['Opp'].replace('Buffalo Bills', 'Bills')
    df['Opp'] = df['Opp'].replace('Tennessee Titans', 'Titans')
    df['Opp'] = df['Opp'].replace('San Francisco 49ers', '49ers')

    df['Base_Team'] = df['Base_Team'].replace(['Los Angeles Rams', 'St. Louis Rams'], 'Rams')
    df['Base_Team'] = df['Base_Team'].replace(['Los Angeles Chargers', 'San Diego Chargers'], 'Chargers')
    df['Base_Team'] = df['Base_Team'].replace('Seattle Seahawks', 'Seahawks')
    df['Base_Team'] = df['Base_Team'].replace('Arizona Cardinals', 'Cardinals')
    df['Base_Team'] = df['Base_Team'].replace('Green Bay Packers', 'Packers')
    df['Base_Team'] = df['Base_Team'].replace('New Orleans Saints', 'Saints')
    df['Base_Team'] = df['Base_Team'].replace('New York Giants', 'Giants')
    df['Base_Team'] = df['Base_Team'].replace('Chicago Bears', 'Bears')
    df['Base_Team'] = df['Base_Team'].replace('Carolina Panthers', 'Panthers')
    df['Base_Team'] = df['Base_Team'].replace('Tampa Bay Buccaneers', 'Buccaneers')
    df['Base_Team'] = df['Base_Team'].replace('Atlanta Falcons', 'Falcons')
    df['Base_Team'] = df['Base_Team'].replace('Washington Redskins', 'Washington')
    df['Base_Team'] = df['Base_Team'].replace('Baltimore Ravens', 'Ravens')
    df['Base_Team'] = df['Base_Team'].replace('Philadelphia Eagles', 'Eagles')
    df['Base_Team'] = df['Base_Team'].replace('Dallas Cowboys', 'Cowboys')
    df['Base_Team'] = df['Base_Team'].replace('Kansas City Chiefs', 'Chiefs')
    df['Base_Team'] = df['Base_Team'].replace('Detroit Lions', 'Lions')
    df['Base_Team'] = df['Base_Team'].replace('Minnesota Vikings', 'Vikings')
    df['Base_Team'] = df['Base_Team'].replace('Cincinnati Bengals', 'Bengals')
    df['Base_Team'] = df['Base_Team'].replace('Oakland Raiders', 'Raiders')
    df['Base_Team'] = df['Base_Team'].replace('Pittsburgh Steelers', 'Steelers')
    df['Base_Team'] = df['Base_Team'].replace('Cleveland Browns', 'Browns')
    df['Base_Team'] = df['Base_Team'].replace('Denver Broncos', 'Broncos')
    df['Base_Team'] = df['Base_Team'].replace('Indianapolis Colts', 'Colts')
    df['Base_Team'] = df['Base_Team'].replace('Houston Texans', 'Texans')
    df['Base_Team'] = df['Base_Team'].replace('New York Jets', 'Jets')
    df['Base_Team'] = df['Base_Team'].replace('Jacksonville Jaguars', 'Jaguars')
    df['Base_Team'] = df['Base_Team'].replace('Miami Dolphins', 'Dolphins')
    df['Base_Team'] = df['Base_Team'].replace('New England Patriots', 'Patriots')
    df['Base_Team'] = df['Base_Team'].replace('Buffalo Bills', 'Bills')
    df['Base_Team'] = df['Base_Team'].replace('Tennessee Titans', 'Titans')
    df['Base_Team'] = df['Base_Team'].replace('San Francisco 49ers', '49ers')
    
    return df

### Import Data

In [646]:
allteam_df = pd.read_csv('Resources/allteam_csvs/allteam_df.csv')

In [647]:
allteam_df = clean_team_names(df=allteam_df)

## Update with new week data

In [648]:
allteam_df = allteam_df[allteam_df['Year'] < 2020]

In [649]:
team_df = clean_excel(team='cardinals')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Kliff Kingsbury', 
                        OC='Kliff Kingsbury', 
                        DC='Vance Joseph', 
                        QB='Kyler Murray')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Cardinals'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [650]:
allteam_df[allteam_df['Year'] == 2020]

,Year,Week,Day,Base_Team,Opp,Home,After_Bye,Wins Tally,PF Tally,PA Tally,...,TO_lost Tally,DPassY Tally,DRushY Tally,DTotYd Tally,TO_won Tally,Head_Coach,OC,DC,QB,Results
5120,2020,1.0,Sun,Cardinals,49ers,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray,1.0
5121,2020,2.0,Sun,Cardinals,Washington,1.0,0,1.0,24.0,20.0,...,1.0,243.0,123.0,366.0,0.0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray,1.0
5122,2020,3.0,Sun,Cardinals,Lions,1.0,0,2.0,54.0,35.0,...,2.0,442.0,240.0,682.0,2.0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray,0.0
5123,2020,4.0,Sun,Cardinals,Panthers,0.0,0,2.0,77.0,61.0,...,5.0,674.0,330.0,1004.0,2.0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray,0.0
5124,2020,5.0,Sun,Cardinals,Jets,0.0,0,2.0,98.0,92.0,...,6.0,950.0,498.0,1448.0,3.0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray,1.0
5125,2020,6.0,Mon,Cardinals,Cowboys,0.0,0,3.0,128.0,102.0,...,7.0,1112.0,621.0,1733.0,3.0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray,0.0
5126,2020,7.0,Sun,Cardinals,Seahawks,1.0,0,3.0,128.0,102.0,...,7.0,1112.0,621.0,1733.0,3.0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray,0.0
5127,2020,9.0,Sun,Cardinals,Dolphins,1.0,1,3.0,128.0,102.0,...,7.0,1112.0,621.0,1733.0,3.0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray,0.0
5128,2020,10.0,Sun,Cardinals,Bills,1.0,0,3.0,128.0,102.0,...,7.0,1112.0,621.0,1733.0,3.0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray,0.0
5129,2020,11.0,Thu,Cardinals,Seahawks,0.0,0,3.0,128.0,102.0,...,7.0,1112.0,621.0,1733.0,3.0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray,0.0


In [651]:
team_df = clean_excel(team='falcons')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Dan Quinn', HC2='Raheem Morris', HC2_range=range(6,18,1), 
                        OC='Dirk Koetter', 
                        DC='Raheem Morris', 
                        QB='Matt Ryan')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Falcons'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [652]:
team_df = clean_excel(team='ravens')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='John Harbaugh', 
                        OC='Greg Roman', 
                        DC='Don Martindale', 
                        QB='Lamar Jackson')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Ravens'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [653]:
team_df = clean_excel(team='bills')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Sean McDermott', 
                        OC='Brian Daboll', 
                        DC='Leslie Frazier', 
                        QB='Josh Allen')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Bills'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [654]:
team_df = clean_excel(team='panthers')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Matt Rhule', 
                        OC='Joe Brady', 
                        DC='Phil Snow', 
                        QB='Teddy Bridgewater')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Panthers'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [655]:
team_df = clean_excel(team='bears')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Matt Nagy', 
                        OC='Bill Lazor', 
                        DC='Chuck Pagano', 
                        QB='Mitchell Trubisky', 
                        QB2='Nick Foles', QB2_range=range(4,18,1))
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Bears'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [656]:
team_df = clean_excel(team='bengals')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Zac Taylor', 
                        OC='Brian Callahan', 
                        DC='Lou Anarumo', 
                        QB='Joe Burrow')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Bengals'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [657]:
team_df = clean_excel(team='browns')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Kevin Stefanski', 
                        OC='Alex Van Pelt', 
                        DC='Joe Woods', 
                        QB='Baker Mayfield')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Browns'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [658]:
team_df = clean_excel(team='cowboys')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Mike McCarthy',
                        OC='Kellen Moore', 
                        DC='Mike Nolan', 
                        QB='Dak Prescott')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Cowboys'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [659]:
team_df = clean_excel(team='broncos')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Vic Fangio', 
                        OC='Pat Shurmur', 
                        DC='Ed Donatell', 
                        QB='Drew Lock', 
                        QB2='Jeff Driskel', QB2_range=3)
team_df.loc[team_df['Week'] == 4, 'QB'] = "Brett Rypien"
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Broncos'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [660]:
team_df = clean_excel(team='lions')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Matt Patricia', 
                        OC='Darrell Bevell', 
                        DC='Cory Undlin', 
                        QB='Matthew Stafford')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Lions'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [661]:
team_df = clean_excel(team='packers')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Matt LaFleur', 
                        OC='Nathaniel Hackett', 
                        DC='Mike Pettine', 
                        QB='Aaron Rodgers')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Packers'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [662]:
team_df = clean_excel(team='texans')

team_df = yearly_data(df=team_df, year=2020, 
                        HC="Bill O'Brien", HC2_range=range(5,18,1), HC2='Romeo Crennel', 
                        OC='Tim Kelly', 
                        DC='Anthony Weaver', 
                        QB='Deshaun Watson')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Texans'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [663]:
team_df = clean_excel(team='colts')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Frank Reich', 
                        OC='Nick Sirianni', 
                        DC='Matt Eberflus', 
                        QB='Philip Rivers')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Colts'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [664]:
team_df = clean_excel(team='jaguars')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Doug Marrone', 
                        OC='Jay Gruden', 
                        DC='Todd Wash', 
                        QB='Gardner Minshew')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Jaguars'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [665]:
team_df = clean_excel(team='chiefs')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Andy Reid', 
                        OC='Eric Bieniemy', 
                        DC='Steve Spagnuolo', 
                        QB='Patrick Mahomes')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Chiefs'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [666]:
team_df = clean_excel(team='chargers')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Anthony Lynn', 
                        OC='Shane Steichen', 
                        DC='Gus Bradley', 
                        QB='Tyrod Taylor', 
                        QB2='Justin Herbert', QB2_range=range(2,7,1))
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Chargers'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [667]:
team_df = clean_excel(team='rams')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Sean McVay', 
                        OC="Kevin O'Connell", 
                        DC='Brandon Staley', 
                        QB='Jared Goff')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Rams'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [668]:
team_df = clean_excel(team='dolphins')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Brian Flores', 
                        OC='Chan Gailey', 
                        DC='Josh Boyer', 
                        QB='Ryan Fitzpatrick')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Dolphins'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [669]:
team_df = clean_excel(team='vikings')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Mike Zimmer', 
                        OC='Gary Kubiak', 
                        DC='Andre Patterson', 
                        QB='Kirk Cousins')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Vikings'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [670]:
team_df = clean_excel(team='patriots')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Bill Belichick', 
                        QB='Cam Newton', 
                        QB2='Brian Hoyer', QB2_range=(4,6,1))
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Patriots'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [671]:
team_df = clean_excel(team='saints')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC='Dennis Allen', 
                        QB='Drew Brees')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Saints'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [672]:
team_df = clean_excel(team='giants')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Joe Judge', 
                        OC='Jason Garrett', 
                        DC='Patrick Graham', 
                        QB='Daniel Jones')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Giants'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [673]:
team_df = clean_excel(team='jets')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Adam Gase', 
                        OC='Dowell Loggains', 
                        DC='Gregg Williams', 
                        QB='Sam Darnold', QB2='Joe Flacco', QB2_range=range(5,7,1))
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Jets'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [674]:
team_df = clean_excel(team='raiders')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Jon Gruden', 
                        OC='Greg Olson', 
                        DC='Paul Guenther', 
                        QB='Derek Carr')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Raiders'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [675]:
team_df = clean_excel(team='eagles')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Doug Pederson', 
                        OC='Doug Pederson', 
                        DC='Jim Schwartz', 
                        QB='Carson Wentz')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Eagles'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [676]:
team_df = clean_excel(team='steelers')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Mike Tomlin', 
                        OC='Randy Fichtner', 
                        DC='Keith Butler', 
                        QB='Ben Roethlisberger')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Steelers'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [677]:
team_df = clean_excel(team='niners')

team_df = yearly_data(df=team_df, year=2020, 
                      HC='Kyle Shanahan', 
                      OC='Kyle Shanahan', 
                      DC='Robert Saleh', 
                      QB='Jimmy Garoppolo', 
                      QB2='Nick Mullens', QB2_range=range(3,5,1))
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = '49ers'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [678]:
team_df = clean_excel(team='seahawks')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Pete Carroll', 
                        OC='Brian Schottenheimer', 
                        DC='Ken Norton', 
                        QB='Russell Wilson')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Seahawks'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [679]:
team_df = clean_excel(team='bucs')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Bruce Arians', 
                        OC='Byron Leftwich', 
                        DC='Todd Bowles', 
                        QB='Tom Brady')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Buccaneers'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [680]:
team_df = clean_excel(team='titans')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Mike Vrabel', 
                        OC='Arthur Smith', 
                        DC='Mike Vrabel', 
                        QB='Ryan Tannehill')
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Titans'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [681]:
team_df = clean_excel(team='wash')

team_df = yearly_data(df=team_df, year=2020, 
                        HC='Ron Rivera', 
                        OC='Scott Turner', 
                        DC='Jack Del Rio', 
                        QB='Dwayne Haskins', 
                        QB2='Kyle Allen', QB2_range=range(5,7,1))
#team_df.loc[team_df['Week'] == 6, 'HC'] = ""
#team_df.loc[team_df['Week'] == 6, 'OC'] = ""
#team_df.loc[team_df['Week'] == 6, 'DC'] = ""
#team_df.loc[team_df['Week'] == 6, 'QB'] = ""

team_df['Base_Team'] = 'Washington'
team_df = team_df.sort_values(by=['Year', 'Week'], ascending=[False, True])
team_df = clean_team_names(df=team_df)
allteam_df = allteam_df.append(team_df, ignore_index=True)

In [682]:
allteam_df

,Year,Week,Day,Base_Team,Opp,Home,After_Bye,Wins Tally,PF Tally,PA Tally,...,TO_lost Tally,DPassY Tally,DRushY Tally,DTotYd Tally,TO_won Tally,Head_Coach,OC,DC,QB,Results
0,2019,1.0,Sun,49ers,Buccaneers,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,1.0
1,2019,2.0,Sun,49ers,Bengals,0.0,0,1.0,31.0,17.0,...,2.0,174.0,121.0,295.0,4.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,1.0
2,2019,3.0,Sun,49ers,Steelers,1.0,0,2.0,72.0,34.0,...,3.0,465.0,146.0,611.0,5.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,1.0
3,2019,5.0,Mon,49ers,Browns,1.0,1,3.0,96.0,54.0,...,8.0,625.0,225.0,850.0,7.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,1.0
4,2019,6.0,Sun,49ers,Rams,0.0,0,4.0,127.0,57.0,...,8.0,703.0,327.0,1030.0,11.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5627,2020,13.0,Sun,Washington,Steelers,0.0,0,1.0,89.0,142.0,...,8.0,1134.0,648.0,1782.0,7.0,Ron Rivera,Scott Turner,Jack Del Rio,Dwayne Haskins,0.0
5628,2020,14.0,Sun,Washington,49ers,0.0,0,1.0,89.0,142.0,...,8.0,1134.0,648.0,1782.0,7.0,Ron Rivera,Scott Turner,Jack Del Rio,Dwayne Haskins,0.0
5629,2020,15.0,Sun,Washington,Seahawks,1.0,0,1.0,89.0,142.0,...,8.0,1134.0,648.0,1782.0,7.0,Ron Rivera,Scott Turner,Jack Del Rio,Dwayne Haskins,0.0
5630,2020,16.0,Sun,Washington,Panthers,1.0,0,1.0,89.0,142.0,...,8.0,1134.0,648.0,1782.0,7.0,Ron Rivera,Scott Turner,Jack Del Rio,Dwayne Haskins,0.0


In [683]:
# Clean up column names

allteam_df = allteam_df[['Year', 'Week', 
            'Day', 'Base_Team', 
            'Opp', 'Home', 
            'After_Bye', 'Wins Tally', 
            'PF Tally', 'PA Tally', 
            'OPassY Tally', 'ORushY Tally',
            'OTotYd Tally', 'TO_lost Tally', 
            'DPassY Tally', 'DRushY Tally', 
            'DTotYd Tally', 'TO_won Tally',
            'Head_Coach', 'OC', 'DC', 'QB', 
            'Results']]

allteam_df = allteam_df[allteam_df['Week'].isnull() == False]

# Drop weeks that haven't happened yet so they don't mess with the stats
drop_index = allteam_df[(allteam_df['Year'] == 2020) & (allteam_df['Week'] > 6)].index
allteam_df.drop(drop_index, inplace=True)

In [684]:
allteam_df = allteam_df.sort_values(by=['Base_Team','Year', 'Week'], ascending=[True, False, True])

In [685]:
allteam_df = allteam_df.reset_index(drop=True)
allteam_df

,Year,Week,Day,Base_Team,Opp,Home,After_Bye,Wins Tally,PF Tally,PA Tally,...,TO_lost Tally,DPassY Tally,DRushY Tally,DTotYd Tally,TO_won Tally,Head_Coach,OC,DC,QB,Results
0,2020,1.0,Sun,49ers,Cardinals,1.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,0.0
1,2020,2.0,Sun,49ers,Jets,0.0,0,0.0,20.0,24.0,...,0.0,224.0,180.0,404.0,1.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,1.0
2,2020,3.0,Sun,49ers,Giants,0.0,0,1.0,51.0,37.0,...,1.0,397.0,284.0,681.0,1.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,1.0
3,2020,4.0,Sun,49ers,Eagles,1.0,0,2.0,87.0,46.0,...,1.0,562.0,350.0,912.0,4.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,0.0
4,2020,5.0,Sun,49ers,Dolphins,1.0,0,2.0,107.0,71.0,...,4.0,736.0,443.0,1179.0,5.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5297,2010,13.0,Sun,Washington,Giants,0.0,0,5.0,215.0,262.0,...,16.0,2963.0,1447.0,4410.0,21.0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb,0.0
5298,2010,14.0,Sun,Washington,Buccaneers,1.0,0,5.0,222.0,293.0,...,22.0,3124.0,1644.0,4768.0,22.0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb,0.0
5299,2010,15.0,Sun,Washington,Cowboys,0.0,0,5.0,238.0,310.0,...,23.0,3386.0,1747.0,5133.0,24.0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb,0.0
5300,2010,16.0,Sun,Washington,Jaguars,0.0,0,5.0,268.0,343.0,...,26.0,3686.0,1881.0,5567.0,24.0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb,1.0


In [686]:
allteam_df.loc[(allteam_df['Week'] == 6) & (allteam_df['Year'] == 2020) & (allteam_df['Base_Team'] == 'Chiefs'), 'Day'] = "Mon"
allteam_df.loc[(allteam_df['Week'] == 6) & (allteam_df['Year'] == 2020) & (allteam_df['Base_Team'] == 'Bills'), 'Day'] = "Mon"

In [687]:
allteam_df[(allteam_df['Year'] == 2020) & (allteam_df['Base_Team'] == 'Chargers')]

,Year,Week,Day,Base_Team,Opp,Home,After_Bye,Wins Tally,PF Tally,PA Tally,...,TO_lost Tally,DPassY Tally,DRushY Tally,DTotYd Tally,TO_won Tally,Head_Coach,OC,DC,QB,Results
1327,2020,1.0,Sun,Chargers,Bengals,0.0,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,Anthony Lynn,Shane Steichen,Gus Bradley,Tyrod Taylor,1.0
1328,2020,2.0,Sun,Chargers,Chiefs,1.0,0,1.0,16.0,13.0,...,0.0,173.0,122.0,295.0,2.0,Anthony Lynn,Shane Steichen,Gus Bradley,Tyrod Taylor,0.0
1329,2020,3.0,Sun,Chargers,Panthers,1.0,0,1.0,36.0,36.0,...,1.0,462.0,247.0,709.0,2.0,Anthony Lynn,Shane Steichen,Gus Bradley,Tyrod Taylor,0.0
1330,2020,4.0,Sun,Chargers,Buccaneers,0.0,0,1.0,52.0,57.0,...,5.0,683.0,328.0,1011.0,2.0,Anthony Lynn,Shane Steichen,Gus Bradley,Tyrod Taylor,0.0
1331,2020,5.0,Mon,Chargers,Saints,0.0,0,1.0,83.0,95.0,...,7.0,1052.0,443.0,1495.0,3.0,Anthony Lynn,Shane Steichen,Gus Bradley,Tyrod Taylor,0.0


In [688]:
allteam_df[(allteam_df['Year'] == 2020) & (allteam_df['Week'] == 6) & (allteam_df['Home'] == 1)]

,Year,Week,Day,Base_Team,Opp,Home,After_Bye,Wins Tally,PF Tally,PA Tally,...,TO_lost Tally,DPassY Tally,DRushY Tally,DTotYd Tally,TO_won Tally,Head_Coach,OC,DC,QB,Results
5,2020,6.0,Sun,49ers,Rams,1.0,0,2.0,124.0,114.0,...,7.0,1078.0,537.0,1615.0,5.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo,0.0
503,2020,6.0,Mon,Bills,Chiefs,1.0,0,4.0,139.0,142.0,...,8.0,1316.0,543.0,1859.0,6.0,Sean McDermott,Brian Daboll,Leslie Frazier,Josh Allen,0.0
1000,2020,6.0,Sun,Buccaneers,Packers,1.0,0,3.0,139.0,112.0,...,7.0,1199.0,292.0,1491.0,9.0,Bruce Arians,Byron Leftwich,Todd Bowles,Tom Brady,0.0
1663,2020,6.0,Sun,Colts,Bengals,1.0,0,3.0,126.0,88.0,...,5.0,898.0,432.0,1330.0,9.0,Frank Reich,Nick Sirianni,Matt Eberflus,Philip Rivers,0.0
1829,2020,6.0,Mon,Cowboys,Cardinals,1.0,0,2.0,163.0,180.0,...,11.0,1243.0,779.0,2022.0,3.0,Mike McCarthy,Kellen Moore,Mike Nolan,Dak Prescott,0.0
1995,2020,6.0,Sun,Dolphins,Jets,1.0,0,2.0,136.0,113.0,...,5.0,1268.0,630.0,1898.0,8.0,Brian Flores,Chan Gailey,Josh Boyer,Ryan Fitzpatrick,0.0
2161,2020,6.0,Sun,Eagles,Ravens,1.0,0,1.0,113.0,145.0,...,11.0,1205.0,571.0,1776.0,5.0,Doug Pederson,Doug Pederson,Jim Schwartz,Carson Wentz,0.0
2493,2020,6.0,Sun,Giants,Washington,1.0,0,0.0,81.0,133.0,...,9.0,1162.0,553.0,1715.0,6.0,Joe Judge,Jason Garrett,Patrick Graham,Daniel Jones,0.0
2659,2020,6.0,Sun,Jaguars,Lions,1.0,0,1.0,109.0,147.0,...,7.0,1401.0,683.0,2084.0,5.0,Doug Marrone,Jay Gruden,Todd Wash,Gardner Minshew,0.0
3321,2020,6.0,Sun,Panthers,Bears,1.0,0,3.0,122.0,118.0,...,5.0,1111.0,667.0,1778.0,8.0,Matt Rhule,Joe Brady,Phil Snow,Teddy Bridgewater,0.0


In [689]:
week1 = allteam_df[allteam_df['Week'] == 1]
week2 = allteam_df[allteam_df['Week'] == 2]
week3 = allteam_df[allteam_df['Week'] == 3]
week4 = allteam_df[allteam_df['Week'] == 4]
week5 = allteam_df[allteam_df['Week'] == 5]
week6 = allteam_df[allteam_df['Week'] == 6]
week7 = allteam_df[allteam_df['Week'] == 7]
week8 = allteam_df[allteam_df['Week'] == 8]
week9 = allteam_df[allteam_df['Week'] == 9]
week10 = allteam_df[allteam_df['Week'] == 10]
week11 = allteam_df[allteam_df['Week'] == 11]
week12 = allteam_df[allteam_df['Week'] == 12]
week13 = allteam_df[allteam_df['Week'] == 13]
week14 = allteam_df[allteam_df['Week'] == 14]
week15 = allteam_df[allteam_df['Week'] == 15]
week16 = allteam_df[allteam_df['Week'] == 16]
week17 = allteam_df[allteam_df['Week'] == 17]

In [690]:
week1.to_csv('Resources/weekly_csvs/week1.csv', index=False)
week2.to_csv('Resources/weekly_csvs/week2.csv', index=False)
week3.to_csv('Resources/weekly_csvs/week3.csv', index=False)
week4.to_csv('Resources/weekly_csvs/week4.csv', index=False)
week5.to_csv('Resources/weekly_csvs/week5.csv', index=False)
week6.to_csv('Resources/weekly_csvs/week6.csv', index=False)
week7.to_csv('Resources/weekly_csvs/week7.csv', index=False)
week8.to_csv('Resources/weekly_csvs/week8.csv', index=False)
week9.to_csv('Resources/weekly_csvs/week9.csv', index=False)
week10.to_csv('Resources/weekly_csvs/week10.csv', index=False)
week11.to_csv('Resources/weekly_csvs/week11.csv', index=False)
week12.to_csv('Resources/weekly_csvs/week12.csv', index=False)
week13.to_csv('Resources/weekly_csvs/week13.csv', index=False)
week14.to_csv('Resources/weekly_csvs/week14.csv', index=False)
week15.to_csv('Resources/weekly_csvs/week15.csv', index=False)
week16.to_csv('Resources/weekly_csvs/week16.csv', index=False)
week17.to_csv('Resources/weekly_csvs/week17.csv', index=False)
allteam_df.to_csv('Resources/allteam_csvs/allteam_df_updated.csv', index=False)

### Prep for Site

In [691]:
# Prep for PickSys Site (user-friendly) using a COPY of the allteam set
allteam_df_current = allteam_df.copy()
allteam_df_current['Results'] = allteam_df_current['Results'].replace(0, 'Lost')
allteam_df_current['Results'] = allteam_df_current['Results'].replace(1, 'Won')
allteam_df_current['Home'] = allteam_df_current['Home'].replace(0, 'Away')
allteam_df_current['Home'] = allteam_df_current['Home'].replace(1, 'Home')
allteam_df_current['After_Bye'] = allteam_df_current['After_Bye'].replace(0, 'No')
allteam_df_current['After_Bye'] = allteam_df_current['After_Bye'].replace(1, 'Yes')

allteam_df_current = allteam_df_current[['Results', 'Year', 'Week', 
            'Day', 'Base_Team', 
            'Opp', 'Home', 
            'After_Bye', 'Wins Tally', 
            'PF Tally', 'PA Tally', 
            'OPassY Tally', 'ORushY Tally',
            'OTotYd Tally', 'TO_lost Tally', 
            'DPassY Tally', 'DRushY Tally', 
            'DTotYd Tally', 'TO_won Tally',
            'Head_Coach', 'OC', 'DC', 'QB']]

allteam_df_current = allteam_df_current.dropna()
allteam_df_current

,Results,Year,Week,Day,Base_Team,Opp,Home,After_Bye,Wins Tally,PF Tally,...,OTotYd Tally,TO_lost Tally,DPassY Tally,DRushY Tally,DTotYd Tally,TO_won Tally,Head_Coach,OC,DC,QB
0,Lost,2020,1.0,Sun,49ers,Cardinals,Home,No,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
1,Won,2020,2.0,Sun,49ers,Jets,Away,No,0.0,20.0,...,366.0,0.0,224.0,180.0,404.0,1.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
2,Won,2020,3.0,Sun,49ers,Giants,Away,No,1.0,51.0,...,725.0,1.0,397.0,284.0,681.0,1.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
3,Lost,2020,4.0,Sun,49ers,Eagles,Home,No,2.0,87.0,...,1145.0,1.0,562.0,350.0,912.0,4.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
4,Lost,2020,5.0,Sun,49ers,Dolphins,Home,No,2.0,107.0,...,1562.0,4.0,736.0,443.0,1179.0,5.0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5297,Lost,2010,13.0,Sun,Washington,Giants,Away,No,5.0,215.0,...,3660.0,16.0,2963.0,1447.0,4410.0,21.0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb
5298,Lost,2010,14.0,Sun,Washington,Buccaneers,Home,No,5.0,222.0,...,3998.0,22.0,3124.0,1644.0,4768.0,22.0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb
5299,Lost,2010,15.0,Sun,Washington,Cowboys,Away,No,5.0,238.0,...,4397.0,23.0,3386.0,1747.0,5133.0,24.0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb
5300,Won,2010,16.0,Sun,Washington,Jaguars,Away,No,5.0,268.0,...,4738.0,26.0,3686.0,1881.0,5567.0,24.0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb


In [692]:
allteam_df_current.to_csv('Resources/allteam_csvs/allteam_df_picksys.csv', index=False)